Team 10 NLP Project

Dataset: One Million Post Corpus
Dataset Language: German

Goal: Classify and predict User-Post catgeory


In [12]:
import sqlite3
import pandas as pd
# Create your connection.
cnx = sqlite3.connect('corpus.sqlite3')

df = pd.read_sql_query("SELECT * FROM Annotations", cnx)
print(df.head())

df2 = pd.read_sql_query("SELECT * FROM Annotations_consolidated", cnx)
print(df2.head())

df3 = pd.read_sql_query("SELECT * FROM Articles", cnx)
print(df3.head())

df4 = pd.read_sql_query("SELECT * FROM Categories", cnx)
print(df4.head())

df5 = pd.read_sql_query("SELECT * FROM CrossValSplit", cnx)
print(df5.head())

df6 = pd.read_sql_query("SELECT * FROM Newspaper_Staff", cnx)
print(df6.head())

df7 = pd.read_sql_query("SELECT * FROM Posts", cnx)
print(df7.head())

   ID_Post  ID_Annotator         Category  Value
0     3326             1    ArgumentsUsed      0
1     3326             1   Discriminating      0
2     3326             1    Inappropriate      0
3     3326             1         OffTopic      0
4     3326             1  PersonalStories      0
   ID_Post          Category  Value
0       79  PossiblyFeedback      1
1       81  PossiblyFeedback      1
2      132  PossiblyFeedback      1
3      134  PossiblyFeedback      1
4      139  PossiblyFeedback      1
   ID_Article                              Path          publishingDate  \
0           1           Newsroom/User/Community  2012-05-26 03:00:19.23   
1           2    Newsroom/User/Community/Regeln  2012-05-26 12:12:19.46   
2           3                    Diverses/mobil  2013-11-22 12:15:00.00   
3           4  Newsroom/User/mitmachen/Mitreden  2014-08-13 05:30:00.00   
4           5  Newsroom/User/mitmachen/Mitreden  2014-08-27 12:27:01.09   

                                       

In [ ]:
def getData(db_name):
    # open and (if not exists) create database file
    con = sqlite3.connect(db_name)
    cur = con.cursor()

    # create table
    cur.execute("CREATE TABLE IF NOT EXISTS Posts_Annotated (" +
                "ID_Post           INTEGER PRIMARY KEY," +
                "Status            INTEGER," +
                "Headline          TEXT," +
                "Body              TEXT," +
                "PositiveVotes     INTEGER," +
                "NegativeVotes     INTEGER," +
                "ArgumentsUsed     INTEGER," +
                "Discriminating    INTEGER," +
                "Inappropriate     INTEGER," +
                "OffTopic          INTEGER," +
                "PersonalStories   INTEGER," +
                "PossiblyFeedback  INTEGER," +
                "SentimentNegative INTEGER," +
                "SentimentNeutral  INTEGER," +
                "SentimentPositive INTEGER" +
                ");")

    # insert statement
    insert_table = "INSERT INTO Posts_Annotated ( ID_Post, Status, Headline, Body, PositiveVotes, NegativeVotes, ArgumentsUsed, Discriminating, Inappropriate, OffTopic, PersonalStories, PossiblyFeedback, SentimentNegative, SentimentNeutral, SentimentPositive ) values ( ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ? )"

    # select annotation
    get_annotation = "SELECT Category, Value FROM Annotations_consolidated WHERE ID_Post = ? ORDER BY Category"

    # get all posts we need
    cur.execute('SELECT ID_Post, ' +
                'Status, ' +
                'Headline, ' +
                'Body, ' +
                'PositiveVotes, ' +
                'NegativeVotes ' +
                'FROM ( ' +
                'SELECT * ' +
                'FROM Posts ' +
                'WHERE ID_Post IN ( ' +
                'SELECT ID_Post ' +
                'FROM Annotations_consolidated ' +
                'WHERE Category = "ArgumentsUsed"));')

    rows = cur.fetchall()

    for row in rows:
        next_row = []

        for row_data in row:
            next_row.append(row_data)

        # get annotations
        cur.execute(get_annotation, [row[0]])
        annotations = cur.fetchall()
        for annotation in annotations:
            next_row.append(annotation[1])

        cur.execute(insert_table, next_row)

    # Save (commit) the changes
    con.commit()

    # close the database connection
    con.close()

getData("corpus.sqlite3")